# Visualization notebook



## Imports and functions

In [7]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
import spacy
from spacy import Language
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# 1,071,477 authors

# Exploring MUD

In [90]:
import ijson
import sqlite3
import random

data = ijson.parse(open("data/mud/full/raw_mud/raw_all/data.jsonl"),multiple_values=True)

x = 0
d = {}
posts = []
for prefix, event, value in data:
    
    if prefix == "syms.item":
        posts.append(value)
        
    if prefix.startswith("author_id"):
        d[value] = posts
        posts = []
        x += 1

    if x == 30:
        break



author = random.choice(list(d.keys()))


save = {author:d[author]}
print(len(d[author]))

utils.save_json(save, "example_author.json")

In [23]:
@Language.component("boundary_syms")
def boundary_sym_pipe(doc):
    for token in doc:
        if token.text == "$":
            token.pos_ = "[BOS]"
        elif token.text == "[EOS]":
            token.pos_ = "[EOS]"
    return doc



nlp = spacy.load("en_core_web_md")
nlp.add_pipe("boundary_syms", after="tagger")

nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7fcf5b7ed060>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7fcf5b7ecfa0>),
 ('boundary_syms', <function __main__.boundary_sym_pipe(doc)>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fcf6dcd7840>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7fcf5c4cf640>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7fcf610d0a00>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fcf6dcd77d0>)]

In [24]:
text = "$ This is a string!!! [EOS]"
doc = nlp(text)

for token in doc:
    print(token.text, token.pos_)

ValueError: [E1021] `pos` value "[BOS]" is not a valid Universal Dependencies tag. Non-UD tags should use the `tag` property.